In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In [3]:
rnnamp_model = pd.read_csv('/kaggle/input/hemolytic/combined.csv')

In [4]:
X=rnnamp_model['text']
y=np.array(rnnamp_model['labels'])


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam,RMSprop,SGD
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
from keras.saving import register_keras_serializable


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=0)


In [7]:
'''
# Example data
texts = X_train

# Tokenize the texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# Convert text to sequences of integers
sequences = tokenizer.texts_to_sequences(texts)
test_seq=tokenizer.texts_to_sequences(X_test)
# Pad sequences to a fixed length
maxlen = 100  # Adjust as needed based on your data
padded_sequences = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')
test=pad_sequences(test_seq, maxlen=maxlen, padding='post', truncating='post')
# Convert to NumPy array
X_train = tf.constant(padded_sequences)
X_test = tf.constant(test)
'''

"\n# Example data\ntexts = X_train\n\n# Tokenize the texts\ntokenizer = Tokenizer()\ntokenizer.fit_on_texts(texts)\n\n# Convert text to sequences of integers\nsequences = tokenizer.texts_to_sequences(texts)\ntest_seq=tokenizer.texts_to_sequences(X_test)\n# Pad sequences to a fixed length\nmaxlen = 100  # Adjust as needed based on your data\npadded_sequences = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')\ntest=pad_sequences(test_seq, maxlen=maxlen, padding='post', truncating='post')\n# Convert to NumPy array\nX_train = tf.constant(padded_sequences)\nX_test = tf.constant(test)\n"

In [8]:
X_train=np.array(X_train)
X_test=np.array(X_test)

In [9]:
def tokenizer(text,max_len):
  #dic={'A':1,'G':1,'V':1,'I':2,'L':2,'F':2,'P':2,'Y':3,'M':3,'T':3,'S':3,'H':4,'N':4,'Q':4,'W':4,'K':5,'R':5,'D':6,'E':6,'C':7}
  dic={'A':1,'G':2,'V':3,'I':4,'L':5,'F':6,'P':7,'Y':8,'M':9,'T':10,'S':11,'H':12,'N':13,'Q':14,'W':15,'K':16,'R':17,'D':18,'E':19,'C':20}
  onehot=[]
  t=[]
  for i in range(len(text)):
    row=[]
    l=[]
    char=text[i].split(' ')
    for j in range(max_len):
      if j< len(char):
        row.append(dic[char[j]])
        r=np.zeros(20)
        r[dic[char[j]]-1]=1
      else:
        r=np.ones(20)*-1
        row.append(0)
      l.append(r)
    l=np.array(l)
    onehot.append(l)
    t.append(row)
  onehot=np.array(onehot)
  t=np.array(t)
  return t,onehot
max_len=50
X_train,onehot_train=tokenizer(X_train,max_len)
X_test,onehot_test=tokenizer(X_test,max_len)


In [10]:
def positional_encoding(positions, d):

    # initialize a matrix angle_rads of all the angles
    pos=np.arange(positions)[:, np.newaxis] #Column vector containing the position span [0,1,..., positions]
    k= np.arange(d)[np.newaxis, :]  #Row vector containing the dimension span [[0, 1, ..., d-1]]
    i = k//2
    angle_rads = pos/(10000**(2*i/d)) #Matrix of angles indexed by (pos,i)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    #adds batch axis
    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)


In [11]:
@register_keras_serializable()
class TransformerModel(keras.Model):
    def __init__(self, input_vocab_size, d_model, num_heads, ff_dim, rate=0.1, maxlen=50):
        super(TransformerModel, self).__init__()

        self.embedding = layers.Embedding(input_vocab_size, d_model)
        self.PE = positional_encoding(maxlen, d_model)
        self.transformer_block = TransformerBlock_Encode(d_model, num_heads, ff_dim, rate)
        self.transformer_block2 = TransformerBlock_decode(d_model, num_heads, ff_dim, rate)
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(512, activation="relu")
        self.fc3 = layers.Dense(256, activation="relu")
        self.fc2 = layers.Dense(1, activation="sigmoid")

    def call(self, inputs, training):
        x = self.embedding(inputs)
        #x = x+self.PE
        y = self.transformer_block(x)
        x = self.transformer_block2(x,y,y)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc3(x)
        return self.fc2(x)
class TransformerBlock_decode(layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock_decode, self).__init__()

        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.att1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(d_model)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-1)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-1)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-1)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)
    def call(self, inputs,q,k, training):
        attn_output = self.att(inputs, inputs,inputs)
        out1 = self.layernorm1(inputs + attn_output)
        attn_output1=self.att(q, k,out1)
        out2 = self.layernorm1(out1 + attn_output1)
        ffn_output = self.ffn(out2)
        return self.layernorm2(out2 + ffn_output)

# Define the TransformerBlock layer
class TransformerBlock_Encode(layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock_Encode, self).__init__()
        self.con= layers.Conv1D(256,5,padding='same')
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(d_model)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-1)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-1)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        inputs=self.con(inputs)
        attn_output = self.att(inputs, inputs,inputs)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + ffn_output)


# Define your data loading and preprocessing here
# Example: X_train, y_train = load_data_and_preprocess()

# Define the model
input_vocab_size = 1024  # Replace with the actual vocabulary size
d_model = 256
num_heads = 8
ff_dim = 256

model = TransformerModel(input_vocab_size, d_model, num_heads, ff_dim)
initial_learning_rate = 0.0001
optimizer = Adam(learning_rate=initial_learning_rate)
# Compile the model
model.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=["accuracy"])

callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),tf.keras.callbacks.ModelCheckpoint(filepath='AMAP1.h5', monitor='val_accuracy', save_best_only=True,mode='auto',save_weights_only=True)]

# Train the model
history = model.fit(X_train,y_train,epochs = 100,batch_size=32,validation_split=0.1,callbacks=[callback],verbose=1,shuffle= True)

Epoch 1/100
162/162 [==============================] - 17s 42ms/step - loss: 0.4707 - accuracy: 0.7759 - val_loss: 0.3767 - val_accuracy: 0.8574
Epoch 2/100
162/162 [==============================] - 6s 38ms/step - loss: 0.3194 - accuracy: 0.8578 - val_loss: 0.3498 - val_accuracy: 0.8609
Epoch 3/100
162/162 [==============================] - 6s 38ms/step - loss: 0.2574 - accuracy: 0.8913 - val_loss: 0.3510 - val_accuracy: 0.8643
Epoch 4/100
162/162 [==============================] - 6s 38ms/step - loss: 0.2148 - accuracy: 0.9129 - val_loss: 0.3312 - val_accuracy: 0.8748
Epoch 5/100
162/162 [==============================] - 6s 38ms/step - loss: 0.1772 - accuracy: 0.9307 - val_loss: 0.3245 - val_accuracy: 0.8765
Epoch 6/100
162/162 [==============================] - 6s 38ms/step - loss: 0.1535 - accuracy: 0.9375 - val_loss: 0.3285 - val_accuracy: 0.8817
Epoch 7/100
162/162 [==============================] - 6s 38ms/step - loss: 0.1282 - accuracy: 0.9499 - val_loss: 0.3610 - val_accuracy

In [12]:
model.load_weights('/kaggle/input/amap/tensorflow2/model/1/AMAP1.h5')

In [13]:
model1=model

In [14]:
from sklearn import model_selection, metrics
y_pred=model.predict(X_train)
fpr, tpr, thresholds = metrics.roc_curve(y_train, y_pred, pos_label=1)
auc=metrics.auc(fpr, tpr)

y_pred[y_pred>0.5]=1
y_pred[y_pred<0.5]=0

cv_preds = y_pred
print('combined train datasets')
name='deep learning'
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_train, cv_preds)))
print("%s: Precision-Recall %0.2f%%" % (name, 100*metrics.average_precision_score(y_train, cv_preds)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*metrics.matthews_corrcoef(y_train, cv_preds)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_train, cv_preds)))
print("%s: F1-Score %0.2f%%" % (name, 100*metrics.f1_score(y_train, cv_preds)))
print("%s: AUC Score %0.2f%%" % (name, 100*auc))
target_names = ['low 0', 'high 1']
print(metrics.classification_report(y_train, cv_preds, target_names=target_names))

# Predictions Validation Set
print('combined test datasets')
y_pred2=model.predict(X_test)
l=np.zeros(len(y_pred2))
l=l.reshape(-1,1)
l[y_pred2>=0.5]=1
l[y_pred2<0.5]=0
cv_preds2 = l
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_test, cv_preds2)))
print("%s: Precision %0.2f%%" % (name, 100*metrics.precision_score(y_test, cv_preds2)))
print("%s: Recall %0.2f%%" % (name, 100*metrics.recall_score(y_test, cv_preds2)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*metrics.matthews_corrcoef(y_test, cv_preds2)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_test, cv_preds2)))
print("%s: F1-Score %0.2f%%" % (name, 100*metrics.f1_score(y_test, cv_preds2)))
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred2, pos_label=1)
auc=metrics.auc(fpr, tpr)
print("%s: AUC Score %0.2f%%" % (name, 100*auc))

target_names = ['low 0', 'high 1']
print(metrics.classification_report(y_test, cv_preds2, target_names=target_names))

180/180 [==============================] - 3s 14ms/step
combined train datasets
deep learning: Accuracy 97.54%
deep learning: Precision-Recall 95.46%
deep learning: Matthews Coefficient 94.96%
deep learning: Cohen Kappa Score 94.96%
deep learning: F1-Score 97.08%
deep learning: AUC Score 99.46%
              precision    recall  f1-score   support

       low 0       0.98      0.98      0.98      3332
      high 1       0.97      0.97      0.97      2411

    accuracy                           0.98      5743
   macro avg       0.97      0.97      0.97      5743
weighted avg       0.98      0.98      0.98      5743

combined test datasets
45/45 [==============================] - 1s 14ms/step
deep learning: Accuracy 89.28%
deep learning: Precision 87.59%
deep learning: Recall 86.41%
deep learning: Matthews Coefficient 77.88%
deep learning: Cohen Kappa Score 77.87%
deep learning: F1-Score 86.99%
deep learning: AUC Score 94.23%
              precision    recall  f1-score   support

       

In [15]:
rnnamp_model = pd.read_csv('/kaggle/input/hemolytic/hlppredfuse.csv')

In [16]:
X=rnnamp_model['text']
y=np.array(rnnamp_model['labels'])


In [17]:
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=0)


In [18]:
X_train=np.array(X_train)
X_test=np.array(X_test)

In [19]:
def tokenizer(text,max_len):
  #dic={'A':1,'G':1,'V':1,'I':2,'L':2,'F':2,'P':2,'Y':3,'M':3,'T':3,'S':3,'H':4,'N':4,'Q':4,'W':4,'K':5,'R':5,'D':6,'E':6,'C':7}
  dic={'A':1,'G':2,'V':3,'I':4,'L':5,'F':6,'P':7,'Y':8,'M':9,'T':10,'S':11,'H':12,'N':13,'Q':14,'W':15,'K':16,'R':17,'D':18,'E':19,'C':20}
  onehot=[]
  t=[]
  for i in range(len(text)):
    row=[]
    l=[]
    char=text[i].split(' ')
    for j in range(max_len):
      if j< len(char):
        row.append(dic[char[j]])
        r=np.zeros(20)
        r[dic[char[j]]-1]=1
      else:
        r=np.ones(20)*-1
        row.append(0)
      l.append(r)
    l=np.array(l)
    onehot.append(l)
    t.append(row)
  onehot=np.array(onehot)
  t=np.array(t)
  return t,onehot
max_len=50
X_train,onehot_train=tokenizer(X_train,max_len)
X_test,onehot_test=tokenizer(X_test,max_len)


In [20]:
@register_keras_serializable()
class TransformerModel(keras.Model):
    def __init__(self, input_vocab_size, d_model, num_heads, ff_dim, rate=0.1, maxlen=50):
        super(TransformerModel, self).__init__()

        self.embedding = layers.Embedding(input_vocab_size, d_model)
        self.PE = positional_encoding(maxlen, d_model)
        self.transformer_block = TransformerBlock_Encode(d_model, num_heads, ff_dim, rate)
        self.transformer_block2 = TransformerBlock_decode(d_model, num_heads, ff_dim, rate)
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(512, activation="relu")
        self.fc3 = layers.Dense(256, activation="relu")
        self.fc2 = layers.Dense(1, activation="sigmoid")

    def call(self, inputs, training):
        x = self.embedding(inputs)
        #x = x+self.PE
        y = self.transformer_block(x)
        x = self.transformer_block2(x,y,y)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc3(x)
        return self.fc2(x)
class TransformerBlock_decode(layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock_decode, self).__init__()

        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.att1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(d_model)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-1)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-1)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-1)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)
    def call(self, inputs,q,k, training):
        attn_output = self.att(inputs, inputs,inputs)
        out1 = self.layernorm1(inputs + attn_output)
        attn_output1=self.att(q, k,out1)
        out2 = self.layernorm1(out1 + attn_output1)
        ffn_output = self.ffn(out2)
        return self.layernorm2(out2 + ffn_output)

# Define the TransformerBlock layer
class TransformerBlock_Encode(layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock_Encode, self).__init__()
        self.con= layers.Conv1D(256,5,padding='same')
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(d_model)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-1)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-1)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        inputs=self.con(inputs)
        attn_output = self.att(inputs, inputs,inputs)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + ffn_output)


# Define your data loading and preprocessing here
# Example: X_train, y_train = load_data_and_preprocess()

# Define the model
input_vocab_size = 1024  # Replace with the actual vocabulary size
d_model = 256
num_heads = 8
ff_dim = 256
model2 = TransformerModel(input_vocab_size, d_model, num_heads, ff_dim)
initial_learning_rate = 0.0001
optimizer = Adam(learning_rate=initial_learning_rate)
# Compile the model
model2.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=["accuracy"])

callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),tf.keras.callbacks.ModelCheckpoint(filepath='AMAP2.h5', monitor='val_accuracy', save_best_only=True,mode='auto',save_weights_only=True)]

# Train the model
history = model2.fit(X_train,y_train,epochs = 100,batch_size=32,validation_split=0.1,callbacks=[callback],verbose=1,shuffle= True)

Epoch 1/100
80/80 [==============================] - 12s 52ms/step - loss: 0.3332 - accuracy: 0.8693 - val_loss: 0.1700 - val_accuracy: 0.9397
Epoch 2/100
80/80 [==============================] - 3s 40ms/step - loss: 0.1436 - accuracy: 0.9463 - val_loss: 0.2534 - val_accuracy: 0.9007
Epoch 3/100
80/80 [==============================] - 3s 41ms/step - loss: 0.1292 - accuracy: 0.9534 - val_loss: 0.1774 - val_accuracy: 0.9397
Epoch 4/100
80/80 [==============================] - 3s 39ms/step - loss: 0.0899 - accuracy: 0.9668 - val_loss: 0.1650 - val_accuracy: 0.9326
Epoch 5/100
80/80 [==============================] - 3s 41ms/step - loss: 0.0575 - accuracy: 0.9834 - val_loss: 0.1796 - val_accuracy: 0.9433
Epoch 6/100
80/80 [==============================] - 3s 40ms/step - loss: 0.0418 - accuracy: 0.9870 - val_loss: 0.1578 - val_accuracy: 0.9433
Epoch 7/100
80/80 [==============================] - 3s 40ms/step - loss: 0.0311 - accuracy: 0.9913 - val_loss: 0.1570 - val_accuracy: 0.9433
Epoch

In [21]:
model2.load_weights('/kaggle/input/amap/tensorflow2/model/1/AMAP2.h5')

In [22]:
model2.save_weights("AMAP2.h5")

In [23]:
from sklearn import model_selection, metrics
y_pred=model2.predict(X_train)
y_pred[y_pred>0.5]=1
y_pred[y_pred<0.5]=0

cv_preds = y_pred
print('hlppredfuse train datasets')
name='deep learning'
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_train, cv_preds)))
print("%s: Precision-Recall %0.2f%%" % (name, 100*metrics.average_precision_score(y_train, cv_preds)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*metrics.matthews_corrcoef(y_train, cv_preds)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_train, cv_preds)))
print("%s: F1-Score %0.2f%%" % (name, 100*metrics.f1_score(y_train, cv_preds)))
print("%s: AUC Score %0.2f%%" % (name, 100*auc))
target_names = ['low 0', 'high 1']
print(metrics.classification_report(y_train, cv_preds, target_names=target_names))

# Predictions Validation Set
print('hlppredfuse test datasets')
y_pred2=model2.predict(X_test)
l=np.zeros(len(y_pred2))
l=l.reshape(-1,1)
l[y_pred2>=0.5]=1
l[y_pred2<0.5]=0
cv_preds2 = l
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_test, cv_preds2)))
print("%s: Precision %0.2f%%" % (name, 100*metrics.precision_score(y_test, cv_preds2)))
print("%s: Recall %0.2f%%" % (name, 100*metrics.recall_score(y_test, cv_preds2)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*metrics.matthews_corrcoef(y_test, cv_preds2)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_test, cv_preds2)))
print("%s: F1-Score %0.2f%%" % (name, 100*metrics.f1_score(y_test, cv_preds2)))
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred2, pos_label=1)
auc=metrics.auc(fpr, tpr)
print("%s: AUC Score %0.2f%%" % (name, 100*auc))

target_names = ['low 0', 'high 1']
print(metrics.classification_report(y_test, cv_preds2, target_names=target_names))

88/88 [==============================] - 2s 14ms/step
hlppredfuse train datasets
deep learning: Accuracy 99.29%
deep learning: Precision-Recall 98.29%
deep learning: Matthews Coefficient 98.34%
deep learning: Cohen Kappa Score 98.34%
deep learning: F1-Score 98.85%
deep learning: AUC Score 94.23%
              precision    recall  f1-score   support

       low 0       0.99      1.00      0.99      1938
      high 1       1.00      0.98      0.99       876

    accuracy                           0.99      2814
   macro avg       0.99      0.99      0.99      2814
weighted avg       0.99      0.99      0.99      2814

hlppredfuse test datasets
22/22 [==============================] - 0s 13ms/step
deep learning: Accuracy 96.16%
deep learning: Precision 93.27%
deep learning: Recall 94.55%
deep learning: Matthews Coefficient 91.11%
deep learning: Cohen Kappa Score 91.11%
deep learning: F1-Score 93.91%
deep learning: AUC Score 97.64%
              precision    recall  f1-score   support

   

In [24]:
rnnamp_model = pd.read_csv('/kaggle/input/hemolytic/rnnamp.csv')

In [25]:
X=rnnamp_model['text']
y=np.array(rnnamp_model['labels'])


In [26]:
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=0)


In [27]:
X_train=np.array(X_train)
X_test=np.array(X_test)

In [28]:
def tokenizer(text,max_len):
  #dic={'A':1,'G':1,'V':1,'I':2,'L':2,'F':2,'P':2,'Y':3,'M':3,'T':3,'S':3,'H':4,'N':4,'Q':4,'W':4,'K':5,'R':5,'D':6,'E':6,'C':7}
  dic={'A':1,'G':2,'V':3,'I':4,'L':5,'F':6,'P':7,'Y':8,'M':9,'T':10,'S':11,'H':12,'N':13,'Q':14,'W':15,'K':16,'R':17,'D':18,'E':19,'C':20}
  onehot=[]
  t=[]
  for i in range(len(text)):
    row=[]
    l=[]
    char=text[i].split(' ')
    for j in range(max_len):
      if j< len(char):
        row.append(dic[char[j]])
        r=np.zeros(20)
        r[dic[char[j]]-1]=1
      else:
        r=np.ones(20)*-1
        row.append(0)
      l.append(r)
    l=np.array(l)
    onehot.append(l)
    t.append(row)
  onehot=np.array(onehot)
  t=np.array(t)
  return t,onehot
max_len=50
X_train,onehot_train=tokenizer(X_train,max_len)
X_test,onehot_test=tokenizer(X_test,max_len)


In [29]:
# Define the Transformer model
@register_keras_serializable()
class TransformerModel(keras.Model):
    def __init__(self, input_vocab_size, d_model, num_heads, ff_dim, rate=0.1, maxlen=50):
        super(TransformerModel, self).__init__()

        self.embedding = layers.Embedding(input_vocab_size, d_model)
        self.PE = positional_encoding(maxlen, d_model)
        self.transformer_block = TransformerBlock_Encode(d_model, num_heads, ff_dim, rate)
        self.transformer_block2 = TransformerBlock_decode(d_model, num_heads, ff_dim, rate)
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(512, activation="relu")
        self.fc3 = layers.Dense(256, activation="relu")
        self.fc2 = layers.Dense(1, activation="sigmoid")

    def call(self, inputs, training):
        x = self.embedding(inputs)
        #x = x+self.PE
        y = self.transformer_block(x)
        x = self.transformer_block2(x,y,y)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc3(x)
        return self.fc2(x)
class TransformerBlock_decode(layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock_decode, self).__init__()

        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.att1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(d_model)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-1)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-1)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-1)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)
    def call(self, inputs,q,k, training):
        attn_output = self.att(inputs, inputs,inputs)
        out1 = self.layernorm1(inputs + attn_output)
        attn_output1=self.att(q, k,out1)
        out2 = self.layernorm1(out1 + attn_output1)
        ffn_output = self.ffn(out2)
        return self.layernorm2(out2 + ffn_output)

# Define the TransformerBlock layer
class TransformerBlock_Encode(layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock_Encode, self).__init__()
        self.con= layers.Conv1D(256,5,padding='same')
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(d_model)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-1)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-1)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        inputs=self.con(inputs)
        attn_output = self.att(inputs, inputs,inputs)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + ffn_output)


# Define your data loading and preprocessing here
# Example: X_train, y_train = load_data_and_preprocess()

# Define the model
input_vocab_size = 1024  # Replace with the actual vocabulary size
d_model = 256
num_heads = 8
ff_dim = 256
model3 = TransformerModel(input_vocab_size, d_model, num_heads, ff_dim)
initial_learning_rate = 0.0001
optimizer = Adam(learning_rate=initial_learning_rate)
# Compile the model
model3.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=["accuracy"])
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),tf.keras.callbacks.ModelCheckpoint(filepath='AMAP3.h5', monitor='val_accuracy', save_best_only=True,mode='auto',save_weights_only=True)]

# Train the model
history = model3.fit(X_train,y_train,epochs = 100,batch_size=32,validation_split=0.1,callbacks=[callback],verbose=1,shuffle= True)

Epoch 1/100
58/58 [==============================] - 11s 53ms/step - loss: 0.6422 - accuracy: 0.6679 - val_loss: 0.5675 - val_accuracy: 0.7073
Epoch 2/100
58/58 [==============================] - 2s 38ms/step - loss: 0.4797 - accuracy: 0.7745 - val_loss: 0.5685 - val_accuracy: 0.7073
Epoch 3/100
58/58 [==============================] - 2s 38ms/step - loss: 0.4231 - accuracy: 0.8120 - val_loss: 0.5962 - val_accuracy: 0.7073
Epoch 4/100
58/58 [==============================] - 2s 41ms/step - loss: 0.3699 - accuracy: 0.8462 - val_loss: 0.5294 - val_accuracy: 0.7415
Epoch 5/100
58/58 [==============================] - 2s 39ms/step - loss: 0.3308 - accuracy: 0.8652 - val_loss: 0.5388 - val_accuracy: 0.7366
Epoch 6/100
58/58 [==============================] - 3s 43ms/step - loss: 0.2992 - accuracy: 0.8707 - val_loss: 0.4772 - val_accuracy: 0.7756
Epoch 7/100
58/58 [==============================] - 2s 39ms/step - loss: 0.2451 - accuracy: 0.9000 - val_loss: 0.5342 - val_accuracy: 0.7512
Epoch

In [30]:
model3.load_weights('/kaggle/input/amap/tensorflow2/model/1/AMAP3.h5')

In [31]:
from sklearn import model_selection, metrics
y_pred=model3.predict(X_train)
y_pred[y_pred>0.5]=1
y_pred[y_pred<0.5]=0

cv_preds = y_pred
print('rnnamp train datasets')
name='deep learning'
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_train, cv_preds)))
print("%s: Precision-Recall %0.2f%%" % (name, 100*metrics.average_precision_score(y_train, cv_preds)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*metrics.matthews_corrcoef(y_train, cv_preds)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_train, cv_preds)))
print("%s: F1-Score %0.2f%%" % (name, 100*metrics.f1_score(y_train, cv_preds)))
print("%s: AUC Score %0.2f%%" % (name, 100*auc))
target_names = ['low 0', 'high 1']
print(metrics.classification_report(y_train, cv_preds, target_names=target_names))

# Predictions Validation Set
print('rnnamp test datasets')
y_pred2=model3.predict(X_test)
l=np.zeros(len(y_pred2))
l=l.reshape(-1,1)
l[y_pred2>=0.5]=1
l[y_pred2<0.5]=0
cv_preds2 = l
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_test, cv_preds2)))
print("%s: Precision %0.2f%%" % (name, 100*metrics.precision_score(y_test, cv_preds2)))
print("%s: Recall %0.2f%%" % (name, 100*metrics.recall_score(y_test, cv_preds2)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*metrics.matthews_corrcoef(y_test, cv_preds2)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_test, cv_preds2)))
print("%s: F1-Score %0.2f%%" % (name, 100*metrics.f1_score(y_test, cv_preds2)))
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred2, pos_label=1)
auc=metrics.auc(fpr, tpr)
print("%s: AUC Score %0.2f%%" % (name, 100*auc))

target_names = ['low 0', 'high 1']
print(metrics.classification_report(y_test, cv_preds2, target_names=target_names))


64/64 [==============================] - 1s 16ms/step
rnnamp train datasets
deep learning: Accuracy 94.96%
deep learning: Precision-Recall 93.30%
deep learning: Matthews Coefficient 89.88%
deep learning: Cohen Kappa Score 89.88%
deep learning: F1-Score 95.29%
deep learning: AUC Score 97.64%
              precision    recall  f1-score   support

       low 0       0.95      0.95      0.95       952
      high 1       0.95      0.95      0.95      1093

    accuracy                           0.95      2045
   macro avg       0.95      0.95      0.95      2045
weighted avg       0.95      0.95      0.95      2045

rnnamp test datasets
16/16 [==============================] - 0s 13ms/step
deep learning: Accuracy 79.69%
deep learning: Precision 82.93%
deep learning: Recall 76.69%
deep learning: Matthews Coefficient 59.62%
deep learning: Cohen Kappa Score 59.44%
deep learning: F1-Score 79.69%
deep learning: AUC Score 86.13%
              precision    recall  f1-score   support

       low 0 

In [32]:
model3.save_weights("AMAP3.h5")